In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, regexp_extract, regexp_replace
import pandas as pd

In [2]:
# Create a SparkSession
spark = (SparkSession.builder.appName("Project 2")
         .config("spark.sql.repl.eagerEval.enabled", True)
         .config("spark.sql.parquet.cacheMetadata", "true")
         .config("spark.sql.session.timeZone", "Etc/UTC")
         .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/09/20 18:55:22 WARN Utils: Your hostname, DESKTOP-JJQB7CC resolves to a loopback address: 127.0.1.1; using 172.30.3.30 instead (on interface eth0)
24/09/20 18:55:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/20 18:55:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Read the parquet file 
domain_current = spark.read.parquet('../data/raw/domain_current_listings.parquet')
domain_current.limit(5)

bedrooms,bathrooms,car_parks,url,name,cost_text,type,latitude,longitude
2,1,1,https://www.domai...,705/8 Marmion Pla...,$600 per week,Apartment / Unit ...,-37.8134708,144.9424794
3,2,2,https://www.domai...,5/18-20 Ibbottson...,$650 Per Week,Townhouse,-37.70987239999999,145.0844928
3,2,1,https://www.domai...,2109/35 Malcolm S...,"$1,150/week",Apartment / Unit ...,-37.8369683,144.9964622
2,1,1,https://www.domai...,4/17a The Esplana...,$475.00 per week,Apartment / Unit ...,-38.1345686,144.3548803
3,2,2,https://www.domai...,501/446 Malvern R...,$2200 Per Week,Apartment / Unit ...,-37.8479885,145.0012197


In [4]:
# Count the number of properties
domain_current.count()

14199

In [5]:
# Print the data types of the columns
domain_current.printSchema()

root
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- car_parks: integer (nullable = true)
 |-- url: string (nullable = true)
 |-- name: string (nullable = true)
 |-- cost_text: string (nullable = true)
 |-- type: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)



In [6]:
# Dectect the rows that have missing values in the columns 'name', 'cost_text', 'type', 'latitude' and 'longitude'
domain_current.where(domain_current['name'].isNull()).count(), domain_current.where(domain_current['cost_text'].isNull()).count(), domain_current.where(domain_current['type'].isNull()).count(), domain_current.where(domain_current['latitude'].isNull()).count(), domain_current.where(domain_current['longitude'].isNull()).count()

(0, 0, 0, 0, 0)

In [7]:
# Drop the rows that have missing values in the columns 'bedrooms', 'bathrooms' and 'car_parks'
domain_current = domain_current.dropna(subset=['bedrooms', 'bathrooms', 'car_parks'])
domain_current.count()

14166

In [8]:
# Creat a new column 'suburb' by extracting the suburbs from 'name'
domain_current = domain_current.withColumn('suburb', split(domain_current['name'], ',')[1])
domain_current.limit(5)

bedrooms,bathrooms,car_parks,url,name,cost_text,type,latitude,longitude,suburb
2,1,1,https://www.domai...,705/8 Marmion Pla...,$600 per week,Apartment / Unit ...,-37.8134708,144.9424794,Docklands VIC 3008
3,2,2,https://www.domai...,5/18-20 Ibbottson...,$650 Per Week,Townhouse,-37.70987239999999,145.0844928,Watsonia VIC 3087
3,2,1,https://www.domai...,2109/35 Malcolm S...,"$1,150/week",Apartment / Unit ...,-37.8369683,144.9964622,South Yarra VIC ...
2,1,1,https://www.domai...,4/17a The Esplana...,$475.00 per week,Apartment / Unit ...,-38.1345686,144.3548803,Geelong VIC 3220
3,2,2,https://www.domai...,501/446 Malvern R...,$2200 Per Week,Apartment / Unit ...,-37.8479885,145.0012197,Prahran VIC 3181


In [9]:
# Create a new column 'postcode' by extracting the postcodes from 'suburb'
domain_current = domain_current.withColumn('postcode', regexp_extract(domain_current['suburb'], r'\b(\d{4})\b', 0))
domain_current.limit(5)

bedrooms,bathrooms,car_parks,url,name,cost_text,type,latitude,longitude,suburb,postcode
2,1,1,https://www.domai...,705/8 Marmion Pla...,$600 per week,Apartment / Unit ...,-37.8134708,144.9424794,Docklands VIC 3008,3008
3,2,2,https://www.domai...,5/18-20 Ibbottson...,$650 Per Week,Townhouse,-37.70987239999999,145.0844928,Watsonia VIC 3087,3087
3,2,1,https://www.domai...,2109/35 Malcolm S...,"$1,150/week",Apartment / Unit ...,-37.8369683,144.9964622,South Yarra VIC ...,3141
2,1,1,https://www.domai...,4/17a The Esplana...,$475.00 per week,Apartment / Unit ...,-38.1345686,144.3548803,Geelong VIC 3220,3220
3,2,2,https://www.domai...,501/446 Malvern R...,$2200 Per Week,Apartment / Unit ...,-37.8479885,145.0012197,Prahran VIC 3181,3181


In [10]:
# Create a new column 'rent_pw' by extracting the rents from 'cost_text'
domain_current = domain_current.withColumn('rent_pw', regexp_extract(regexp_replace(domain_current['cost_text'], ',', ''), r'\b(\d+)\b', 0))
domain_current.limit(5)

bedrooms,bathrooms,car_parks,url,name,cost_text,type,latitude,longitude,suburb,postcode,rent_pw
2,1,1,https://www.domai...,705/8 Marmion Pla...,$600 per week,Apartment / Unit ...,-37.8134708,144.9424794,Docklands VIC 3008,3008,600
3,2,2,https://www.domai...,5/18-20 Ibbottson...,$650 Per Week,Townhouse,-37.70987239999999,145.0844928,Watsonia VIC 3087,3087,650
3,2,1,https://www.domai...,2109/35 Malcolm S...,"$1,150/week",Apartment / Unit ...,-37.8369683,144.9964622,South Yarra VIC ...,3141,1150
2,1,1,https://www.domai...,4/17a The Esplana...,$475.00 per week,Apartment / Unit ...,-38.1345686,144.3548803,Geelong VIC 3220,3220,475
3,2,2,https://www.domai...,501/446 Malvern R...,$2200 Per Week,Apartment / Unit ...,-37.8479885,145.0012197,Prahran VIC 3181,3181,2200


In [11]:
# Convert the data type of 'rent_pw' to integer
domain_current = domain_current.withColumn('rent_pw', domain_current['rent_pw'].cast('int'))
domain_current.printSchema()

root
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- car_parks: integer (nullable = true)
 |-- url: string (nullable = true)
 |-- name: string (nullable = true)
 |-- cost_text: string (nullable = true)
 |-- type: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- suburb: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- rent_pw: integer (nullable = true)



In [12]:
# Dectect the rows that have missing values in the column 'rent_pw'
domain_current.where(domain_current['rent_pw'].isNull()).count()

838

In [13]:
# Drop the the rows that have missing values in the column 'rent_pw'
domain_current = domain_current.dropna(subset=['rent_pw'])
domain_current.count()

13328

In [14]:
# Save the cleaned data to a parquet file
domain_current.write.mode('overwrite').parquet('../data/landing/cleaned_domain_current_listings.parquet')

In [15]:
# Aggregate the properties by 'postcode', calculate the average rent per week, and ordered by average rent per week in descending order
avg_rent_per_suburb = domain_current.groupBy('postcode').avg('rent_pw').orderBy('avg(rent_pw)', ascending=False)
# Correct the average rent per week to 2 decimal places
avg_rent_per_suburb = avg_rent_per_suburb.withColumn('avg(rent_pw)', avg_rent_per_suburb['avg(rent_pw)'].cast('decimal(7,2)'))
avg_rent_per_suburb.show(10)


+--------+------------+
|postcode|avg(rent_pw)|
+--------+------------+
|    3699|    39823.33|
|    3730|     6901.56|
|    3722|     2763.40|
|    3428|     1800.00|
|    3564|     1733.07|
|    3953|     1560.60|
|    3233|     1553.89|
|    3944|     1412.50|
|    3186|     1357.15|
|    3113|     1275.00|
+--------+------------+
only showing top 10 rows



In [16]:
# Aggregate the properties by 'type', calculate the average rent per week, and ordered by average rent per week in descending order
avg_rent_per_type = domain_current.groupBy('type').avg('rent_pw').orderBy('avg(rent_pw)', ascending=False)
# Correct the average rent per week to 2 decimal places
avg_rent_per_type = avg_rent_per_type.withColumn('avg(rent_pw)', avg_rent_per_type['avg(rent_pw)'].cast('decimal(6,2)'))
avg_rent_per_type.show()
avg_rent_per_type.count()

+--------------------+------------+
|                type|avg(rent_pw)|
+--------------------+------------+
|    New House & Land|     6768.53|
|Acreage / Semi-Rural|      886.46|
|             Terrace|      794.55|
|              Duplex|      771.25|
|           Townhouse|      766.33|
|New Apartments / ...|      734.60|
|       Semi-Detached|      661.43|
|               House|      651.68|
|Apartment / Unit ...|      637.10|
|               Villa|      571.96|
|      Block of Units|      450.00|
|              Studio|      416.19|
|            Carspace|      202.69|
|            New land|      200.00|
+--------------------+------------+



14

In [17]:
# Read the last page of the xlsx file
historical_rent = pd.read_excel('../data/raw/Moving annual rent by suburb - March quarter 2023.xlsx', sheet_name='All properties')
historical_rent.head()

,Moving annual rent by suburb,Unnamed: 1,Lease commenced in year ending,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 178,Unnamed: 179,Unnamed: 180,Unnamed: 181,Unnamed: 182,Unnamed: 183,Unnamed: 184,Unnamed: 185,Unnamed: 186,Unnamed: 187
0,All properties,NaN,Mar 2000,NaN,Jun 2000,NaN,Sep 2000,NaN,Dec 2000,NaN,...,Mar 2022,NaN,Jun 2022,NaN,Sep 2022,NaN,Dec 2022,NaN,Mar 2023,NaN
1,NaN,NaN,Count,Median,Count,Median,Count,Median,Count,Median,...,Count,Median,Count,Median,Count,Median,Count,Median,Count,Median
2,Inner Melbourne,Albert Park-Middle Park-West St Kilda,1143,260,1134,260,1177,270,1178,275,...,867,500,855,515,881,500,832,525,786,545
3,NaN,Armadale,733,200,737,200,738,205,739,210,...,805,430,851,450,852,450,840,460,751,490
4,NaN,Carlton North,864,260,814,260,799,265,736,270,...,581,580,535,595,547,600,546,600,490,620


In [18]:
# Drop the columns from 'Lease commenced in year ending' to 'Unnamed: 17' (years before 2002 with some missing values) and the fisrt column (irrelevant granular information)
historical_rent = historical_rent.drop(columns=historical_rent.columns[2:18])
historical_rent = historical_rent.drop(columns=historical_rent.columns[0])
historical_rent.head()

,Unnamed: 1,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,...,Unnamed: 178,Unnamed: 179,Unnamed: 180,Unnamed: 181,Unnamed: 182,Unnamed: 183,Unnamed: 184,Unnamed: 185,Unnamed: 186,Unnamed: 187
0,NaN,Mar 2002,NaN,Jun 2002,NaN,Sep 2002,NaN,Dec 2003,NaN,Mar 2003,...,Mar 2022,NaN,Jun 2022,NaN,Sep 2022,NaN,Dec 2022,NaN,Mar 2023,NaN
1,NaN,Count,Median,Count,Median,Count,Median,Count,Median,Count,...,Count,Median,Count,Median,Count,Median,Count,Median,Count,Median
2,Albert Park-Middle Park-West St Kilda,1332,300,1350,300,1415,300,1431,300,1422,...,867,500,855,515,881,500,832,525,786,545
3,Armadale,774,233,783,230,773,230,724,230,747,...,805,430,851,450,852,450,840,460,751,490
4,Carlton North,626,290,647,290,644,290,670,290,685,...,581,580,535,595,547,600,546,600,490,620


In [19]:
# Dorp the even columns (count information)
historical_median = historical_rent.iloc[:, ::2]
historical_median.head()

,Unnamed: 1,Unnamed: 19,Unnamed: 21,Unnamed: 23,Unnamed: 25,Unnamed: 27,Unnamed: 29,Unnamed: 31,Unnamed: 33,Unnamed: 35,...,Unnamed: 169,Unnamed: 171,Unnamed: 173,Unnamed: 175,Unnamed: 177,Unnamed: 179,Unnamed: 181,Unnamed: 183,Unnamed: 185,Unnamed: 187
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Median,Median,Median,Median,Median,Median,Median,Median,Median,...,Median,Median,Median,Median,Median,Median,Median,Median,Median,Median
2,Albert Park-Middle Park-West St Kilda,300,300,300,300,295,290,295,295,300,...,550,500,500,500,495,500,515,500,525,545
3,Armadale,233,230,230,230,235,235,240,240,240,...,495,450,440,425,420,430,450,450,460,490
4,Carlton North,290,290,290,290,290,290,290,290,298,...,590,580,575,575,580,580,595,600,600,620


In [20]:
# Drop the first two rows and reset the index
historical_median = historical_median.drop([0, 1]).reset_index(drop=True)
historical_median.head()

,Unnamed: 1,Unnamed: 19,Unnamed: 21,Unnamed: 23,Unnamed: 25,Unnamed: 27,Unnamed: 29,Unnamed: 31,Unnamed: 33,Unnamed: 35,...,Unnamed: 169,Unnamed: 171,Unnamed: 173,Unnamed: 175,Unnamed: 177,Unnamed: 179,Unnamed: 181,Unnamed: 183,Unnamed: 185,Unnamed: 187
0,Albert Park-Middle Park-West St Kilda,300,300,300,300,295,290,295,295,300,...,550,500,500,500,495,500,515,500,525,545
1,Armadale,233,230,230,230,235,235,240,240,240,...,495,450,440,425,420,430,450,450,460,490
2,Carlton North,290,290,290,290,290,290,290,290,298,...,590,580,575,575,580,580,595,600,600,620
3,Carlton-Parkville,270,270,275,280,280,280,282,284,285,...,395,350,340,330,320,310,319,340,350,400
4,CBD-St Kilda Rd,320,320,320,320,310,310,300,300,300,...,420,380,369,350,350,365,390,419,450,500


In [21]:
# Rename the column 'Unnamed: 19' as '2002-03' and the rest of the columns in the format of 'year-month' as each quarter
historical_median.columns = ['suburb', '2002-03', '2002-06', '2002-09', '2002-12', '2003-03', '2003-06', '2003-09', '2003-12', '2004-03', '2004-06', '2004-09', '2004-12', '2005-03', '2005-06', '2005-09', '2005-12', '2006-03', '2006-06', '2006-09', '2006-12', '2007-03', '2007-06', '2007-09', '2007-12', '2008-03', '2008-06', '2008-09', '2008-12', '2009-03', '2009-06', '2009-09', '2009-12', '2010-03', '2010-06', '2010-09', '2010-12', '2011-03', '2011-06', '2011-09', '2011-12', '2012-03', '2012-06', '2012-09', '2012-12', '2013-03', '2013-06', '2013-09', '2013-12', '2014-03', '2014-06', '2014-09', '2014-12', '2015-03', '2015-06', '2015-09', '2015-12', '2016-03', '2016-06', '2016-09', '2016-12', '2017-03', '2017-06', '2017-09', '2017-12', '2018-03', '2018-06', '2018-09', '2018-12', '2019-03', '2019-06', '2019-09', '2019-12', '2020-03', '2020-06', '2020-09', '2020-12', '2021-03', '2021-06', '2021-09', '2021-12', '2022-03', '2022-06', '2022-09', '2022-12', '2023-03']
historical_median.head()

,suburb,2002-03,2002-06,2002-09,2002-12,2003-03,2003-06,2003-09,2003-12,2004-03,...,2020-12,2021-03,2021-06,2021-09,2021-12,2022-03,2022-06,2022-09,2022-12,2023-03
0,Albert Park-Middle Park-West St Kilda,300,300,300,300,295,290,295,295,300,...,550,500,500,500,495,500,515,500,525,545
1,Armadale,233,230,230,230,235,235,240,240,240,...,495,450,440,425,420,430,450,450,460,490
2,Carlton North,290,290,290,290,290,290,290,290,298,...,590,580,575,575,580,580,595,600,600,620
3,Carlton-Parkville,270,270,275,280,280,280,282,284,285,...,395,350,340,330,320,310,319,340,350,400
4,CBD-St Kilda Rd,320,320,320,320,310,310,300,300,300,...,420,380,369,350,350,365,390,419,450,500


In [22]:
# Drop the rows 'Group Total'
historical_median = historical_median[~historical_median['suburb'].str.contains('Group Total')]

In [23]:
# Spelling correction
historical_median['suburb'] = historical_median['suburb'].replace('Wanagaratta', 'Wangaratta')
historical_median['suburb'] = historical_median['suburb'].replace('Newcombe', 'Newcomb')

In [24]:
# Save it to a csv file
historical_median.to_csv('../data/landing/historical_median_rent_by_suburb.csv', index=False)

In [25]:
# Create a new dataframe which only contains the median rent for the first quarter of each year as the median rent for the year
yearly_median = historical_median[['suburb', '2002-03', '2003-03', '2004-03', '2005-03', '2006-03', '2007-03', '2008-03', '2009-03', '2010-03', '2011-03', '2012-03', '2013-03', '2014-03', '2015-03', '2016-03', '2017-03', '2018-03', '2019-03', '2020-03', '2021-03', '2022-03', '2023-03']]

# Rename the columns
yearly_median.columns = ['suburb', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
yearly_median.head()

,suburb,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Albert Park-Middle Park-West St Kilda,300,295,300,300,313,345,380,410,425,...,460,480,520,520,550,552,600,500,500,545
1,Armadale,233,235,240,240,250,280,320,350,360,...,390,400,400,420,480,480,500,450,430,490
2,Carlton North,290,290,298,300,320,330,380,400,420,...,510,530,530,530,562,580,585,580,580,620
3,Carlton-Parkville,270,280,285,281,275,275,290,310,340,...,355,351,373,395,410,435,404,350,310,400
4,CBD-St Kilda Rd,320,310,300,300,310,340,370,400,410,...,430,440,450,470,495,500,510,380,365,500


In [26]:
# Compute the yearly growth rate of the median rent for each suburb
yearly_growth = pd.DataFrame(yearly_median['suburb'])
for i in range(1, len(yearly_median.columns)-1):
    yearly_growth[str(2002+i)] = ((yearly_median[yearly_median.columns[i+1]] - yearly_median[yearly_median.columns[i]]) / yearly_median[yearly_median.columns[i]]) * 100

# Melt the dataframe to have the columns 'suburb', 'year' and 'growth_rate'
yearly_growth = yearly_growth.melt(id_vars='suburb', var_name='year', value_name='growth_rate')

# Convert the data type of 'year' to integer and 'growth_rate' to float
yearly_growth['year'] = yearly_growth['year'].astype('int')
yearly_growth['growth_rate'] = yearly_growth['growth_rate'].astype('float')
yearly_growth.head()


,suburb,year,growth_rate
0,Albert Park-Middle Park-West St Kilda,2003,-1.666667
1,Armadale,2003,0.858369
2,Carlton North,2003,0.000000
3,Carlton-Parkville,2003,3.703704
4,CBD-St Kilda Rd,2003,-3.125000


In [27]:
# Save it to a csv file
yearly_growth.to_csv('../data/landing/yearly_growth_rate_of_median_rent_by_suburb.csv', index=False)